In [86]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [87]:
# Libraries
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

## Single series

In [88]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0},
    verbose=False
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Train-validation dates
# ==============================================================================
end_train = '2002-01-01 23:59:00'

In [89]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = False,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

# Manual check
# ==============================================================================
mae_foreast = mean_absolute_error(data.loc[end_train:], predictions)
naive_in_sample_foreast = data.loc[:end_train].shift(1).dropna()
mae_in_sample = mean_absolute_error(data.loc[naive_in_sample_foreast.index], naive_in_sample_foreast)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert metric == mae_foreast/mae_in_sample

  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting MASE: 2.344471182248395
mae forecast: 0.19922359134545478
mae in sample: 0.08497591817460318
mase: 2.344471182248395


In [90]:
y_true = data.loc[end_train:]
y_pred = predictions['pred']
y_train = data.loc[:end_train]
mean_absolute_scaled_error(y_true, y_pred, y_train)

2.344471182248395

In [91]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = False,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = 3,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )

print(f"Backtesting MASE: {metric}")

# Manual check
# ==============================================================================
refit_intervals = [
    ('1991-07-01 00:00:00', '2002-01-01 00:00:00'),
    ('1991-07-01 00:00:00', '2004-07-01 00:00:00'),
    ('1991-07-01 00:00:00', '2007-01-01 00:00:00')
]

mae_foreast = mean_absolute_error(data.loc[end_train:], predictions)
y_train = pd.concat([data.loc[start:end] for start, end in refit_intervals])
y_train = y_train.loc[~y_train.index.duplicated(keep='first')]
naive_in_sample_foreast = y_train.shift(1).dropna()
mae_in_sample = mean_absolute_error(y_train.loc[naive_in_sample_foreast.index], naive_in_sample_foreast)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert metric == mae_foreast/mae_in_sample

Information of backtesting process
----------------------------------
Number of observations used for initial training: 127
Number of observations used for backtesting: 77
    Number of folds: 8
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 7 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-02-01 00:00:00 -- 2002-11-01 00:00:00  (n=10)
Fold: 1
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-12-01 00:00:00 -- 2003-09-01 00:00:00  (n=10)
Fold: 2
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2003-10-01 00:00:00 -- 2004-07-01 00:00:00  (n=10)
Fold: 3
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=157)
    Validation: 2004-08-01 00:00:00 -- 2005-05-01 00:00:00  (n=10)
Fold: 4
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=1

  0%|          | 0/8 [00:00<?, ?it/s]

Backtesting MASE: 2.462755601999461
mae forecast: 0.23185816123376632
mae in sample: 0.09414582634408604
mase: 2.462755601999461


## Multiseries

In [92]:
# Download data
# ==============================================================================
data = fetch_dataset(name="items_sales")
data.head()

# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [93]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print("Backtesting MASE:")
display(metric)

# Manual check
# ==============================================================================
mae_foreast = mean_absolute_error(
    data.loc[end_train:], predictions, multioutput="raw_values"
)
naive_in_sample_foreast = data.loc[:end_train].shift(1).dropna().drop_duplicates()
mae_in_sample = mean_absolute_error(
    data.loc[naive_in_sample_foreast.index],
    naive_in_sample_foreast,
    multioutput="raw_values",
)
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mae_foreast/mae_in_sample}")
assert (metric['mean_absolute_scaled_error'] == mae_foreast/mae_in_sample).all()

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

Backtesting MASE:


,levels,mean_absolute_scaled_error
0,item_1,1.091527
1,item_2,1.563339
2,item_3,0.959262


mae forecast: [1.69131952 3.70767238 3.57171715]
mae in sample: [1.54949824 2.37163672 3.72340161]
mase: [1.09152723 1.56333908 0.95926186]


In [94]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data,
                          steps                 = 10,
                          metric                = 'mean_absolute_scaled_error',
                          aggregate_metric      = 'average',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True  
                      )

print("Backtesting MASE:")
display(metric)

# Manual check
# ==============================================================================
mae_foreast = mean_absolute_error(
    data.loc[end_train:], predictions, multioutput="raw_values"
)
naive_in_sample_foreast = data.loc[:end_train].shift(1).dropna().drop_duplicates()
mae_in_sample = mean_absolute_error(
    data.loc[naive_in_sample_foreast.index],
    naive_in_sample_foreast,
    multioutput="raw_values",
)
mase = mae_foreast/mae_in_sample
average_mase = mase.mean()
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mase}")
print(f"average mase: {average_mase}")
assert (metric['mean_absolute_scaled_error'] == average_mase).all()

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

Backtesting MASE:


,levels,mean_absolute_scaled_error
average,"[item_1, item_2, item_3]",1.204709


mae forecast: [1.69131952 3.70767238 3.57171715]
mae in sample: [1.54949824 2.37163672 3.72340161]
mase: [1.09152723 1.56333908 0.95926186]
average mase: 1.2047093909443507


In [95]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )

metric, predictions = backtesting_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data,
                          steps                 = 10,
                          metric                = ['mean_absolute_scaled_error', 'mean_absolute_error'],
                          aggregate_metric      = 'average',
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          gap                   = 0,
                          allow_incomplete_fold = True,
                          refit                 = 10,
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )

print("Backtesting MASE:")
display(metric)

# Manual check
# ==============================================================================
# Manual check
refit_intervals = [
    ('2012-01-01 00:00:00', '2014-07-15 00:00:00'),
    ('2012-04-10 00:00:00', '2014-10-23 00:00:00')
]

mae_foreast = mean_absolute_error(
    data.loc[end_train:], predictions, multioutput="raw_values"
)
data_train = pd.concat([data.loc[start:end, :] for start, end in refit_intervals])
data_train = data_train.loc[~data_train.index.duplicated(keep='first'), :]
naive_in_sample_foreast = data_train.shift(1).dropna()
mae_in_sample = mean_absolute_error(
    data.loc[naive_in_sample_foreast.index],
    naive_in_sample_foreast,
    multioutput="raw_values",
)
mase = mae_foreast/mae_in_sample
average_mase = mase.mean()
print(f"mae forecast: {mae_foreast}")
print(f"mae in sample: {mae_in_sample}")
print(f"mase: {mase}")
print(f"average mase: {average_mase}")
assert (metric['mean_absolute_scaled_error'] == average_mase).all()

Information of backtesting process
----------------------------------
Number of observations used for initial training: 927
Number of observations used for backtesting: 170
    Number of folds: 17
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-07-16 00:00:00 -- 2014-07-25 00:00:00  (n=10)
Fold: 1
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-07-26 00:00:00 -- 2014-08-04 00:00:00  (n=10)
Fold: 2
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-08-05 00:00:00 -- 2014-08-14 00:00:00  (n=10)
Fold: 3
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-08-15 00:00:00 -- 2014-08-24 00:00:00  (n=10)
Fold: 4
    Training:   2012-01-01 00:00:00 -- 2014-07-15 00:00:00  (n=927)
    Validation: 2014-08-25 00:00:00 --

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


  0%|          | 0/17 [00:00<?, ?it/s]

Backtesting MASE:


,levels,mean_absolute_scaled_error,mean_absolute_error
average,"[item_1, item_2, item_3]",1.22805,3.025622


mae forecast: [1.69056888 3.67036308 3.71593437]
mae in sample: [1.52805244 2.33486251 3.69445992]
mase: [1.10635527 1.57198253 1.00581261]
average mase: 1.2280501371035948


In [96]:
# Download data
# ==============================================================================
data = fetch_dataset(name="items_sales")
data.head()

# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
end_val = '2014-10-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_val = data.loc[end_train:end_val, :].copy()
data_test  = data.loc[end_val:, :].copy()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [97]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )
lags_grid =[1, 2, 3]
param_grid = {'alpha': [0, 1, 10000000000]}

results = grid_search_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data.loc[:end_val, :],
                          lags_grid             = lags_grid,
                          param_grid            = param_grid,
                          steps                 = 12,
                          metric                = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                          aggregate_metric      = 'weighted_average',
                          refit                 = False,
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          return_best           = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True
                      )

results.head(4)

9 models compared for 3 level(s). Number of iterations: 9.


/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: All arrays must be of the same length

In [47]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor = Ridge(),
                 lags      = 1 
             )

def search_space(trial):
    search_space  = {
        'lags'             : trial.suggest_categorical('lags', [3, 5]),
        'alpha'            : trial.suggest_float('alpha', 0.1, 1.0),
    }
    
    return search_space

results, best_trial = bayesian_search_forecaster_multiseries(
                          forecaster            = forecaster,
                          series                = data.loc[:end_val, :],
                          search_space          = search_space,
                          steps                 = 12,
                          metric                = 'mean_absolute_error',
                          aggregate_metric      = 'weighted_average',
                          refit                 = False,
                          initial_train_size    = len(data.loc[:end_train]),
                          fixed_train_size      = True,
                          n_trials              = 10,
                          random_state          = 123,
                          return_best           = False,
                          n_jobs                = 'auto',
                          verbose               = False,
                          show_progress         = True,
                          engine                = 'optuna',
                          kwargs_create_study   = {},
                          kwargs_study_optimize = {}
                      )

results.head(4)

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-07-01 16:03:27,368] Trial 0 failed with parameters: {'lags': 3, 'alpha': 0.3041663082077828} because of the following error: The value 'i' could not be cast to float.
[W 2024-07-01 16:03:27,368] Trial 0 failed with value 'item_1'.
[W 2024-07-01 16:03:27,474] Trial 1 failed with parameters: {'lags': 5, 'alpha': 0.4807958141120149} because of the following error: The value 'i' could not be cast to float.
[W 2024-07-01 16:03:27,475] Trial 1 failed with value 'item_1'.
[W 2024-07-01 16:03:27,576] Trial 2 failed with parameters: {'lags': 3, 'alpha': 0.5328387113359249} because of the following error: The value 'i' could not be cast to float.
[W 2024-07-01 16:03:27,577] Trial 2 failed with value 'item_1'.
[W 2024-07-01 16:03:27,665] Trial 3 failed with parameters: {'lags': 3, 'alpha': 0.7561447366456374} because of the following error: The value 'i' could not be cast to float.
[W 2024-07-01 16:03:27,666] Trial 3 failed with value 'item_1'.
[W 2024-07-01 16:03:27,752] Trial 4 failed w

ValueError: No trials are completed yet.